# Create a nanowire with a custom cross-section shape

Create a nanowire from original material by creating a slab (with specific Miller indices) and cutting a custom shape along its Z axis, then, optionally, rotating to align along X axis.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## 1. Prepare the Environment

### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
  
    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages
    await install_packages("")

### 1.2. Set up transformation parameters 

In [ ]:
from typing import List

# Cross-section shape parameters
CENTER_COORDINATE = [0.5, 0.5, 0.5] # Center of the cylinder in units specified by flag below
MAJOR_RADIUS = 0.25 # Cylinder wire radius in units specified by the flag below
MINOR_RADIUS = 0.1 # Cylinder wire radius in units specified by the flag below
USE_CARTESIAN_COORDINATES = False # Flag to use Cartesian coordinates for the center and radii

# Wire parameters 
MILLER_INDICES= (0,0,1)  # Miller indices of the nanowire direction
SUPERCELL_MATRIX = [[9, 0, 0], [0, 9, 0], [0, 0, 30]] # Supercell matrix to cut the cylinder from
VACUUM = 10.0 # Vacuum thickness on the sides in Angstroms
ALIGN_ALONG_X = True

import numpy as np
from mat3ra.made.tools.utils.coordinate import CoordinateCondition

# Custom Coordinate Condition for a hollow cylinder shape
class CustomCoordinateCondition(CoordinateCondition):
    center: List[float] = [0.5, 0.5, 0.5]
    major_radius: float = 0.5
    minor_radius: float = 0.2

    def condition(self, coordinate: List[float]) -> bool:
        coord = np.array(coordinate)
        # Calculate the distance from the center in the xy-plane
        xy_distance = np.linalg.norm(coord[:2] - self.center[:2])
        # Calculate the distance from the torus surface
        distance_to_torus = np.sqrt((xy_distance - self.major_radius) ** 2 + (coord[2] - self.center[2]) ** 2)
        return distance_to_torus <= self.minor_radius
condition = CustomCoordinateCondition(center=CENTER_COORDINATE, major_radius=MAJOR_RADIUS, minor_radius=MINOR_RADIUS).condition

### 1.3. Get input materials

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
material = materials[0] 

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize
visualize(material, repetitions=[3,3,3], rotation="0x")
visualize(material, repetitions=[3,3,3], rotation="-90x")

## 2. Create nanowire
### 2.1. Create a slab and cut a cylinder

In [ ]:
from mat3ra.made.tools.build.supercell import create_supercell
from mat3ra.made.tools.modify import  add_vacuum_sides, filter_by_condition_on_coordinates
from mat3ra.made.tools.build.slab import create_slab, SlabConfiguration

slab_config = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=1,
    vacuum=0,
    use_orthogonal_z=True,
)

slab_unit_cell = create_slab(slab_config)
supercell = create_supercell(slab_unit_cell, supercell_matrix=SUPERCELL_MATRIX)
nanowire_z = filter_by_condition_on_coordinates(supercell, condition=condition, use_cartesian_coordinates=USE_CARTESIAN_COORDINATES)
nanowire_z = add_vacuum_sides(nanowire_z, vacuum=VACUUM, on_x=True, on_y=True)    

### 2.2. Rotate the nanowire to align along X axis

In [ ]:
rotate_90_degree_matrix = [[0, 0, 1], [0, 1, 0], [-1, 0, 0]]
nanowire_x = create_supercell(nanowire_z, supercell_matrix=rotate_90_degree_matrix)
nanowire = nanowire_x if ALIGN_ALONG_X else nanowire_z

## 3. Visualize the result

In [ ]:
visualize(nanowire, repetitions=[1, 1, 1], rotation="0x")
visualize(nanowire, repetitions=[1, 1, 1], rotation="-90x")
visualize(nanowire, repetitions=[1, 1, 1], rotation="-90y")

# 4. Pass material to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials
set_materials(nanowire)